In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import gmaps
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
#Read in CSV
weather_df = pd.read_csv("Output_CSV/weather_df.csv")
weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed
0,0,Soyo,AO,-6.1349,12.3689,42,88,74.37,7.47
1,1,Atuona,PF,-9.8000,-139.0333,36,68,77.85,20.92
2,2,Port Elizabeth,ZA,-33.9180,25.5701,90,100,57.49,16.11
3,3,Kapaa,US,22.0752,-159.3190,40,66,81.55,3.00
4,4,Maierus,RO,45.9000,25.5333,10,74,53.46,1.28
...,...,...,...,...,...,...,...,...,...
604,604,Wahiawa,US,21.5028,-158.0236,75,76,77.47,4.61
605,605,Scarborough,GB,54.2797,-0.4044,95,87,54.45,8.30
606,606,Manzanillo,CU,20.3433,-77.1167,65,72,83.07,3.67
607,607,Wanning,CN,18.8003,110.3967,96,91,80.62,7.54


In [11]:
#Set coordinates for markers and define humidity for heatmap
coordinates = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity']

# Create a heatmap marker layer and display
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(coordinates, weights=humidity, max_intensity=100, point_radius=5, dissipating=False )
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#Revise weather dataframe to cities with only ideal conditions
ideal_weather = weather_df.loc[(weather_df['Temperature'] <84) & (weather_df['Temperature'] >71)
                               & (weather_df['Wind Speed'] < 3) & (weather_df['Cloudiness'] <5)]
    
#Display revised dataframe
ideal_weather

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed
9,9,Noumea,NC,-22.2763,166.4572,1,68,76.98,2.71
157,157,Najran,SA,17.4924,44.1277,2,22,79.45,2.75
247,247,Kohat,PK,33.5820,71.4493,0,58,78.87,2.35
346,346,Minab,IR,27.1467,57.0801,3,47,83.70,2.13
509,509,Koungou,YT,-12.7336,45.2042,0,78,74.91,2.30
543,543,Cayenne,GF,4.9333,-52.3333,0,80,83.08,1.99
578,578,Erzin,TR,36.9559,36.2000,0,76,75.00,1.83


In [5]:
#Make copy of ideal_weather df to avoid splice warning
paradise_df = ideal_weather.copy()

#Create new column in dataframe to hold hotel info
paradise_df['Hotel Name'] = ""
paradise_df["Coordinates"] = tuple(zip(paradise_df['Latitude'], paradise_df['Longitude']))
paradise_df.head(10)

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Hotel Name,Coordinates
9,9,Noumea,NC,-22.2763,166.4572,1,68,76.98,2.71,,"(-22.2763, 166.4572)"
157,157,Najran,SA,17.4924,44.1277,2,22,79.45,2.75,,"(17.4924, 44.1277)"
247,247,Kohat,PK,33.5820,71.4493,0,58,78.87,2.35,,"(33.582, 71.4493)"
346,346,Minab,IR,27.1467,57.0801,3,47,83.70,2.13,,"(27.1467, 57.0801)"
509,509,Koungou,YT,-12.7336,45.2042,0,78,74.91,2.30,,"(-12.7336, 45.2042)"
543,543,Cayenne,GF,4.9333,-52.3333,0,80,83.08,1.99,,"(4.9333, -52.3333)"
578,578,Erzin,TR,36.9559,36.2000,0,76,75.00,1.83,,"(36.9559, 36.2)"


In [6]:
# find the closest hotel to each of the city coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in paradise_df.iterrows():
    params = {
        "location": f'{row[3]},{row[4]}',
        "radius": 5000,
        "type": 'lodging',
        "key": gkey
    }
    response = requests.get(base_url, params=params).json()
    
#Set exception for no results error, otherwise populate Hotel Name column with Hotel Name
    try:
        paradise_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except:
        paradise_df.loc[index, 'Hotel Name'] = "NaN"
        
#Display dataframe with hotels column populated
paradise_df.head(10)

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Hotel Name,Coordinates
9,9,Noumea,NC,-22.2763,166.4572,1,68,76.98,2.71,Hôtel Le Lagon,"(-22.2763, 166.4572)"
157,157,Najran,SA,17.4924,44.1277,2,22,79.45,2.75,محمد هادي ال فطيح,"(17.4924, 44.1277)"
247,247,Kohat,PK,33.5820,71.4493,0,58,78.87,2.35,Millennium Guest House & Restaurant,"(33.582, 71.4493)"
346,346,Minab,IR,27.1467,57.0801,3,47,83.70,2.13,Bam campground,"(27.1467, 57.0801)"
509,509,Koungou,YT,-12.7336,45.2042,0,78,74.91,2.30,Villa Netibor,"(-12.7336, 45.2042)"
543,543,Cayenne,GF,4.9333,-52.3333,0,80,83.08,1.99,Hôtel Le Dronmi,"(4.9333, -52.3333)"
578,578,Erzin,TR,36.9559,36.2000,0,76,75.00,1.83,Artemis Otel,"(36.9559, 36.2)"


In [7]:
#Remove any rows where a hotel was not found
paradise_df.dropna()

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Temperature,Wind Speed,Hotel Name,Coordinates
9,9,Noumea,NC,-22.2763,166.4572,1,68,76.98,2.71,Hôtel Le Lagon,"(-22.2763, 166.4572)"
157,157,Najran,SA,17.4924,44.1277,2,22,79.45,2.75,محمد هادي ال فطيح,"(17.4924, 44.1277)"
247,247,Kohat,PK,33.5820,71.4493,0,58,78.87,2.35,Millennium Guest House & Restaurant,"(33.582, 71.4493)"
346,346,Minab,IR,27.1467,57.0801,3,47,83.70,2.13,Bam campground,"(27.1467, 57.0801)"
509,509,Koungou,YT,-12.7336,45.2042,0,78,74.91,2.30,Villa Netibor,"(-12.7336, 45.2042)"
543,543,Cayenne,GF,4.9333,-52.3333,0,80,83.08,1.99,Hôtel Le Dronmi,"(4.9333, -52.3333)"
578,578,Erzin,TR,36.9559,36.2000,0,76,75.00,1.83,Artemis Otel,"(36.9559, 36.2)"


In [12]:
#Create info box for marker labels
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in paradise_df.iterrows()]

#Set marker locations
marker_locations = paradise_df[['Latitude', 'Longitude']]

#Configure and display map
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))